## Imports

In [41]:
import numpy as np
import pandas as pd
import yfinance as yf
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time

from scipy.stats import spearmanr
from talib import RSI, BBANDS, MACD, ATR

In [42]:
tickers = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'META']
tickers = [ticker.replace('.', '-') for ticker in tickers]  # Replace dots with hyphens for yfinance compatibility
print(tickers)

['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'META']


In [43]:
start_date = '2015-01-01'
end_date = '2025-01-01'

The following method is supposed to download data for each ticket.

In [44]:
def get_data(ticker, start_date, end_date):
    data = yf.download(ticker, start_date, end_date)
    data["Ticker"] = ticker
    return data

In [45]:
os.makedirs("data", exist_ok=True)

print(f"Processing data for {len(tickers)} stocks...")
for ticker in tickers:
    try:
        # Get enhanced data for this stock
        stock_data = get_data(ticker, start_date, end_date)
        
        # Reset index to make Date a column
        stock_data = stock_data.reset_index()

        print(f"Processed {ticker} data with {len(stock_data)} rows.")
        
        # Save individual stock data
        stock_data.to_csv(f"data/{ticker}.csv", index=False)
        
        # Add a small delay to avoid hitting API limits
        time.sleep(2)

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")

/tmp/ipykernel_5779/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed

Processing data for 6 stocks...
Processed AAPL data with 2516 rows.



/tmp/ipykernel_5779/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Processed MSFT data with 2516 rows.


/tmp/ipykernel_5779/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


...

Processed AMZN data with 2516 rows.


/tmp/ipykernel_5779/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Processed GOOGL data with 2516 rows.


/tmp/ipykernel_5779/3049716656.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start_date, end_date)
[*********************100%***********************]  1 of 1 completed


Processed META data with 2516 rows.


Let us analyze how does the dataframe for one stock look like.

In [46]:
files = os.listdir('data') # List of files in the data directory
files

['MSFT.csv', 'AMZN.csv', 'GOOGL.csv', 'NVDA.csv', 'META.csv', 'AAPL.csv']

In [47]:
example_file = 'MSFT.csv'
example_file_path = os.path.join('data', example_file)

In [48]:
msft_data = pd.read_csv(example_file_path)
msft_data.head()

,Date,Close,High,Low,Open,Volume,Ticker
0,NaN,MSFT,MSFT,MSFT,MSFT,MSFT,NaN
1,2015-01-02,39.99869918823242,40.56326583298269,39.810512482054385,39.9131600729795,27913900,MSFT
2,2015-01-05,39.6308708190918,39.97303047791457,39.56243692946277,39.665084500798855,39673900,MSFT
3,2015-01-06,39.04920196533203,39.99014525430785,38.955106983812854,39.67364662264987,36447900,MSFT
4,2015-01-07,39.54533004760742,39.74207259354287,38.91233285861642,39.331479028576595,29114100,MSFT


As we see, there is a redundant second row(row with index 0), and moreover there is redundant indexes in the dataframe. We want to get rid of them and concatenate dataframes.  

In [49]:
dataframes = []
for file in files:
    file_path = os.path.join('data', file)     
    stock_data = pd.read_csv(file_path)
    stock_data.drop([0], inplace=True) # Drop row with index 0
    dataframes.append(stock_data)
if dataframes:
    data = pd.concat(dataframes, ignore_index=True)

In [50]:
data

,Date,Close,High,Low,Open,Volume,Ticker
0,2015-01-02,39.99869918823242,40.56326583298269,39.810512482054385,39.9131600729795,27913900,MSFT
1,2015-01-05,39.6308708190918,39.97303047791457,39.56243692946277,39.665084500798855,39673900,MSFT
2,2015-01-06,39.04920196533203,39.99014525430785,38.955106983812854,39.67364662264987,36447900,MSFT
3,2015-01-07,39.54533004760742,39.74207259354287,38.91233285861642,39.331479028576595,29114100,MSFT
4,2015-01-08,40.70869064331055,40.8455552003686,39.964489818251685,39.990150902978684,29645200,MSFT
...,...,...,...,...,...,...,...
15091,2024-12-24,257.57867431640625,257.58862955018265,254.67565812568793,254.87518901917565,23234700,AAPL
15092,2024-12-26,258.39666748046875,259.47408555117545,257.0100282923795,257.56867823862046,27237100,AAPL
15093,2024-12-27,254.9749298095703,258.0774615569102,252.4510193654514,257.20952960207876,42355300,AAPL
15094,2024-12-30,251.59307861328125,252.88995321263158,250.14657107718884,251.62300520062965,35557500,AAPL


In [51]:
data.dtypes

Date      object
Close     object
High      object
Low       object
Open      object
Volume    object
Ticker    object
dtype: object

The columns should be converted into appropriate datatype. 

In [52]:
data['Date'] = pd.to_datetime(data['Date'])
numeric_columns = ['Close', 'High', 'Low', 'Open', 'Volume']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [53]:
data.dtypes

Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
Ticker            object
dtype: object

In [54]:
data.columns = data.columns.map(lambda x: x.lower())

#### Compute Rolling Average Dollar Volume

In [55]:
data['dollar_vol'] = data[['close', 'volume']].prod(axis=1)

In [56]:
data

,date,close,high,low,open,volume,ticker,dollar_vol
0,2015-01-02,39.998699,40.563266,39.810512,39.913160,27913900,MSFT,1.116520e+09
1,2015-01-05,39.630871,39.973030,39.562437,39.665085,39673900,MSFT,1.572311e+09
2,2015-01-06,39.049202,39.990145,38.955107,39.673647,36447900,MSFT,1.423261e+09
3,2015-01-07,39.545330,39.742073,38.912333,39.331479,29114100,MSFT,1.151327e+09
4,2015-01-08,40.708691,40.845555,39.964490,39.990151,29645200,MSFT,1.206817e+09
...,...,...,...,...,...,...,...,...
15091,2024-12-24,257.578674,257.588630,254.675658,254.875189,23234700,AAPL,5.984763e+09
15092,2024-12-26,258.396667,259.474086,257.010028,257.568678,27237100,AAPL,7.037976e+09
15093,2024-12-27,254.974930,258.077462,252.451019,257.209530,42355300,AAPL,1.079954e+10
15094,2024-12-30,251.593079,252.889953,250.146571,251.623005,35557500,AAPL,8.946021e+09


In [57]:
data['dollar_vol_1m'] = (data.groupby('ticker')['dollar_vol']
                           .rolling(window=21)
                           .mean()).values

In [58]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15096 entries, 0 to 15095
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           15096 non-null  datetime64[ns]
 1   close          15096 non-null  float64       
 2   high           15096 non-null  float64       
 3   low            15096 non-null  float64       
 4   open           15096 non-null  float64       
 5   volume         15096 non-null  int64         
 6   ticker         15096 non-null  object        
 7   dollar_vol     15096 non-null  float64       
 8   dollar_vol_1m  14976 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 1.0+ MB


In [59]:
data['dollar_vol_rank'] = (data.groupby('date')
                             .dollar_vol_1m
                             .rank(ascending=False))

In [60]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15096 entries, 0 to 15095
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             15096 non-null  datetime64[ns]
 1   close            15096 non-null  float64       
 2   high             15096 non-null  float64       
 3   low              15096 non-null  float64       
 4   open             15096 non-null  float64       
 5   volume           15096 non-null  int64         
 6   ticker           15096 non-null  object        
 7   dollar_vol       15096 non-null  float64       
 8   dollar_vol_1m    14976 non-null  float64       
 9   dollar_vol_rank  14976 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 1.2+ MB


### Add some Basic Factors

### Compute the Relative Strength Index

In [61]:
data['rsi'] = data.groupby('ticker')['close'].transform(RSI)

### Compute Bollinger Bands

In [62]:
def compute_bb(close_prices):
    """
    Compute Bollinger Bands for a single ticker's close prices
    Returns a DataFrame with bb_high and bb_low columns
    """
    high, mid, low = BBANDS(close_prices, timeperiod=20)
    return pd.DataFrame({
        'bb_high': high, 
        'bb_low': low
    }, index=close_prices.index)

In [63]:
bb_results = data.groupby('ticker')['close'].apply(compute_bb)
bb_results = bb_results.reset_index(level=0, drop=True) # Reset index to align with original data
data = data.join(bb_results)

In [64]:
data['bb_high'] = data['bb_high'].sub(data['close']).div(data['bb_high']).apply(np.log1p)
data['bb_low'] = data['close'].sub(data['bb_low']).div(data['close']).apply(np.log1p)

### Compute Average True Range

In [65]:
def compute_atr(stock_data):
    """
    Compute normalized ATR for a single ticker
    """
    try:
        df = ATR(stock_data['high'], stock_data['low'], 
                 stock_data['close'], timeperiod=14)
        return df.sub(df.mean()).div(df.std())
    except:
        return pd.Series(np.nan, index=stock_data.index)


In [69]:
data['atr'] = data.groupby('ticker', group_keys=False).apply(compute_atr, include_groups=False)

### Compute Moving Average
### Convergance/Divergance

In [71]:
def compute_macd(close_prices):
    """
    Compute normalized MACD for a single ticker
    """
    try:
        macd = MACD(close_prices)[0]  # Get only the MACD line
        return (macd - np.mean(macd)) / np.std(macd)
    except:
        return pd.Series(np.nan, index=close_prices.index)


In [72]:
data['macd'] = data.groupby('ticker')['close'].apply(compute_macd).reset_index(level=0, drop=True)

### Compute Lagged Returns

In [74]:
lags = [1, 5, 10, 21, 42, 63]

In [75]:
returns = data.groupby('ticker')['close'].pct_change()
percentiles=[.0001, .001, .01]
percentiles+= [1-p for p in percentiles]
returns.describe(percentiles=percentiles).iloc[2:].to_frame('percentiles').style.format(lambda x: f'{x:,.2%}')

,percentiles
std,2.18%
min,-26.39%
0.01%,-21.71%
0.1%,-9.95%
1%,-5.74%
50%,0.12%
99%,6.41%
99.9%,13.92%
99.99%,23.82%
max,29.81%


In [76]:
q = 0.0001

### Winsorize outliers

In [77]:
for lag in lags:
    data[f'return_{lag}d'] = (data.groupby('ticker')['close']
                                .pct_change(lag)
                                .pipe(lambda x: x.clip(lower=x.quantile(q),
                                                       upper=x.quantile(1 - q)))
                                .add(1)
                                .pow(1 / lag)
                                .sub(1)
                                )

### Shift lagged outliers

In [79]:
for t in [1, 2, 3, 4, 5]:
    for lag in [1, 5, 10, 21]:
        data[f'return_{lag}d_lag{t}'] = (data.groupby('ticker')
                                           [f'return_{lag}d'].shift(t * lag))

### Compute forward returns

In [81]:
for t in [1, 5, 10, 21]:
    data[f'target_{t}d'] = data.groupby('ticker')[f'return_{t}d'].shift(-t)

### Create year and month columns

In [84]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [88]:
data.dropna(inplace=True)

In [89]:
data

,date,close,high,low,open,volume,ticker,dollar_vol,dollar_vol_1m,dollar_vol_rank,...,return_1d_lag5,return_5d_lag5,return_10d_lag5,return_21d_lag5,target_1d,target_5d,target_10d,target_21d,year,month
126,2015-07-06,38.490154,38.568193,38.108636,38.117305,23034000,MSFT,8.865822e+08,4.496708e+09,1.0,...,-0.008543,-0.002373,0.004396,-0.005553,-0.002028,0.005128,0.005558,0.003270,2015,7
127,2015-07-07,38.412102,38.576851,37.562354,38.446786,36435800,MSFT,1.399576e+09,4.552907e+09,1.0,...,-0.019664,0.001403,0.013846,-0.004842,-0.001354,0.005890,0.006532,0.003407,2015,7
128,2015-07-08,38.360077,38.932356,38.177985,38.533492,39785900,MSFT,1.526190e+09,4.583439e+09,1.0,...,-0.004958,0.001413,0.014087,-0.003455,0.006330,0.006779,0.002900,0.002498,2015,7
129,2015-07-09,38.602882,39.209847,38.585540,38.802313,32424700,MSFT,1.251687e+09,4.681207e+09,1.0,...,0.006795,-0.003213,0.016716,-0.004098,0.002021,0.009434,0.003515,0.002320,2015,7
130,2015-07-10,38.680908,39.140465,38.646224,39.027742,25465800,MSFT,9.850403e+08,4.788976e+09,1.0,...,-0.001125,-0.004637,0.015032,-0.005528,0.020847,0.008853,0.002942,0.002822,2015,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15070,2024-11-22,229.316818,230.164779,227.511176,227.511176,38168300,AAPL,8.752633e+09,3.117485e+10,1.0,...,-0.014109,0.006464,-0.003219,0.005517,0.013051,0.008317,0.007111,0.005550,2024,11
15071,2024-11-25,232.309601,232.688691,229.187143,230.903006,90152800,AAPL,2.094336e+10,3.202636e+10,1.0,...,0.013422,0.004439,-0.005680,0.005705,0.009404,0.008262,0.006221,0.005081,2024,11
15072,2024-11-26,234.494324,235.003106,232.768491,232.768491,45986200,AAPL,1.078350e+10,3.210548e+10,1.0,...,0.001140,0.001713,-0.002717,0.004845,-0.000553,0.006675,0.004759,0.003995,2024,11
15073,2024-11-27,234.364639,235.122820,233.247339,233.905755,33498400,AAPL,7.850840e+09,3.250429e+10,1.0,...,0.003154,-0.000882,-0.000072,0.005968,0.010216,0.006811,0.005413,0.003384,2024,11


In [90]:
data.to_csv('data/stocks.csv')